## Introducción

En el siguiente proyecto, trabajaré con una base de datos de una empresa de transporte individual, analizaré los datos de los competidores y probaré una hipótesis sobre el impacto del clima en la frecuencia de los viajes realizados.

Mi objetivo es encontrar patrones en la información disponible para comprender las preferencias de los pasajeros y el impacto de los factores externos en los viajes.

## 1. Inicialización

In [1]:
#importar librerías
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mt
from math import factorial
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import requests

## Cargar datos

In [2]:
#leer las bases de datos en dataframes
sql_result_01 = pd.read_csv('datasets/project_sql_result_01.csv')
sql_result_04 = pd.read_csv('datasets/project_sql_result_04.csv')
sql_result_07 = pd.read_csv('datasets/project_sql_result_07.csv')

## 2. Preparar los datos

##### 2.1 Analizando conjunto de datos #01

In [3]:
#imprimir informacion general del conjunto #01
sql_result_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


Tipos de datos correctos en cada columna.

In [4]:
#imprimir muestra de datos del conjunto #01
sql_result_01.head()

,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasing,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299


In [5]:
#buscando duplicados en el conjunto #01
sql_result_01.duplicated().sum()

0

In [6]:
#buscando valores ausentes en el conjunto #01
sql_result_01.isna().sum()

company_name    0
trips_amount    0
dtype: int64

Sin valores ausentes ni duplicados en el conjunto #01.

##### 2.2 Analizando conjunto de datos #04

In [7]:
#imprimir informacion general del conjunto #04
sql_result_04.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB


Tipos de datos correctos en cada columna.

In [8]:
#imprimir muestra de datos del conjunto #04
sql_result_04.head()

,dropoff_location_name,average_trips
0,Loop,10727.466667
1,River North,9523.666667
2,Streeterville,6664.666667
3,West Loop,5163.666667
4,O'Hare,2546.900000


In [9]:
#buscando valores duplicados
sql_result_04.duplicated().sum()

0

In [10]:
#buscando valores ausentes
sql_result_04.isna().sum()

dropoff_location_name    0
average_trips            0
dtype: int64

Sin valores ausentes ni duplicados en el conjunto #04.